In [2]:
import numpy as np
import pandas as pd
import scipy as sc

def getDF(pc, bl, seqs):
    parent_kids = pd.read_csv(pc, delimiter =',', names = ['Parent', 'Child']) # load up the parent child table
    branches = pd.DataFrame({"Length": list(open(bl))[0].split(',')}).astype(float) # load up the branch lengths
    combined = pd.concat([parent_kids, branches], axis = 1) # combine parent child table + branch lengths 

    n = min(parent_kids.loc[:,'Parent']) # smallest parent
    l = range(n-1,0, -1) #list of numbers from the min-1 to 1

    combined = pd.concat([combined, pd.DataFrame({"Sequences" : ["NA"]*len(combined["Parent"])})], axis = 1) # add NA for sequences

    sequences = pd.read_csv(seqs, header = None, delimiter = " ") # load up sequences for terminal nodes
    #sequences = sequences.iloc[::-1] # order from highest to lowest
    sequences.columns =["Child", "Sequence"] # give column names
    sequences["Sequence"] = [list(x) for x in sequences["Sequence"]]
    seql = len(sequences.iloc[1,1]) # sequence length
    
    merged_combined = pd.merge(combined, sequences, on='Child', how='left')
    combined['Sequences'] = merged_combined['Sequence']

    # combine sequences, branch lengths for terminal nodes
    #kids = pd.DataFrame({"Parent" : list(l), "Child" : ["NA"]*len(l), "Lengths" : [0]*len(l), "Sequences" : sequences["Lengths"]})

    # return whole combined df
    #return pd.concat([combined, kids], ignore_index = True, axis = 0), seql
    return combined, seql

tree, seql = getDF("/Users/cmco/Desktop/APP/table.dat", "/Users/cmco/Desktop/APP/branchlength.dat", "/Users/cmco/Desktop/APP/msa.dat")
print(tree)
#print(seql)

m = max(tree.loc[:,'Parent']) # biggest parent
nts = ['A', 'C', 'G', 'T']

def get_vectors(i, m, nts): #i index of nucleotide, m biggest parent, nts nucleotides sequences
    lh_table = pd.DataFrame({"Node" : list(range(1,m+1,1)), "Vector" : m*[[0,0,0,0]], "Likelihood" : m*["NA"]})
    c = [x for x in tree["Sequences"] if type(x) == list] #store sequences != NaN
    
    for y in range(len(c)):
        lh_table.at[y, "Vector"] = [1 if c[y][i] == nts[x] else 0 for x in range(len(nts))]

    return lh_table

lh_table = get_vectors(0, m, nts)
print(lh_table)

mu = 0.3
q = np.array([[-3*mu, mu, mu, mu], [mu, -3*mu, mu, mu], [mu, mu, -3*mu, mu], [mu, mu, mu, -3*mu]])

def comp_lh(q, bl, vector):
    print(q)
    print(bl)
    print(vector)
    log_lh = sc.linalg.expm(q*bl) * np.array(vector)
    print(log_lh)

lh = comp_lh(0.3, tree.at[0, "Length"], lh_table.at[0, "Vector"])

'''
def get_lh(mu, tree, seql):
    subtree = tree.tail(list(tree["Child"]).count("NA"))
    #print(subtree)
    for x in subtree["Sequences"]:
        c = x[0]*4
        a = [x for x in c if c]
        print(a)

    #seql = len(subtree["Sequences"])
    #print(seql)
    
    #for x in subtree:
'''

#print(tree["Sequences"][1][5])
#get_lh(0.3, tree, seql)


   Parent  Child  Length                                          Sequences
0       9      1    0.10  [A, G, A, T, C, A, A, G, A, T, C, A, A, G, A, ...
1       9      2    0.40  [A, G, C, T, C, A, A, G, C, T, C, A, A, G, C, ...
2       8      9    0.01                                                NaN
3       8      3    0.04  [C, G, C, T, A, T, C, G, C, T, A, T, C, G, C, ...
4       7      4    0.20  [C, G, T, T, A, C, C, G, T, T, A, C, C, G, T, ...
5       7      5    0.08  [C, G, C, T, A, C, C, G, C, T, A, C, C, G, C, ...
6       6      7    0.12                                                NaN
7       6      8    0.14                                                NaN
   Node        Vector Likelihood
0     1  [1, 0, 0, 0]         NA
1     2  [1, 0, 0, 0]         NA
2     3  [0, 1, 0, 0]         NA
3     4  [0, 1, 0, 0]         NA
4     5  [0, 1, 0, 0]         NA
5     6  [0, 0, 0, 0]         NA
6     7  [0, 0, 0, 0]         NA
7     8  [0, 0, 0, 0]         NA
8     9  [0, 0, 0, 

'\ndef get_lh(mu, tree, seql):\n    subtree = tree.tail(list(tree["Child"]).count("NA"))\n    #print(subtree)\n    for x in subtree["Sequences"]:\n        c = x[0]*4\n        a = [x for x in c if c]\n        print(a)\n\n    #seql = len(subtree["Sequences"])\n    #print(seql)\n    \n    #for x in subtree:\n'